# Clasificación con Random Forest para Diagnóstico de Cáncer de Mama

Este notebook implementa un modelo de Random Forest para clasificar tumores como malignos o benignos basado en características celulares.

## Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.feature_selection import SelectFromModel
import pickle

## Cargar los datos

In [ ]:
# Cargar los datos
df = pd.read_csv('../data/breast-cancer.csv')

# Exploración inicial
print("Información del dataset:")
df.info()

print("\nDistribución de clases:")
print(df['diagnosis'].value_counts())

## Preparación de datos

In [ ]:
# Convertir la variable objetivo a numérica (M=1, B=0)
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# Eliminar la columna ID que no aporta información para la predicción
df = df.drop('id', axis=1)

# Separar características y variable objetivo
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras, {X_train.shape[1]} características')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]} muestras, {X_test.shape[1]} características')

## Optimización de hiperparámetros

In [ ]:
# Definir la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear el modelo base
rf = RandomForestClassifier(random_state=42)

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entrenar el modelo con búsqueda de cuadrícula
grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Obtener el mejor modelo
best_rf = grid_search.best_estimator_

## Evaluación del modelo

In [ ]:
# Realizar predicciones
y_pred = best_rf.predict(X_test_scaled)
y_prob = best_rf.predict_proba(X_test_scaled)[:, 1]

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud (Accuracy): {accuracy:.4f}')

# Informe de clasificación
print('
Informe de clasificación:')
print(classification_report(y_test, y_pred, target_names=['Benigno', 'Maligno']))

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Benigno', 'Maligno'], yticklabels=['Benigno', 'Maligno'])
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.savefig('../results/breast_cancer_confusion_matrix.png')
plt.show()

## Curva ROC

In [ ]:
# Calcular la curva ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Visualizar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Área bajo la curva = {roc_auc:.4f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.savefig('../results/breast_cancer_roc_curve.png')
plt.show()

## Importancia de características

In [ ]:
# Obtener la importancia de las características
feature_importances = best_rf.feature_importances_

# Crear un DataFrame para visualizar la importancia
importance_df = pd.DataFrame({
    'Característica': X.columns,
    'Importancia': feature_importances
}).sort_values('Importancia', ascending=False)

# Visualizar la importancia de las características
plt.figure(figsize=(12, 8))
sns.barplot(x='Importancia', y='Característica', data=importance_df.head(15))
plt.title('Top 15 Características Más Importantes')
plt.tight_layout()
plt.savefig('../results/breast_cancer_feature_importance.png')
plt.show()

## Selección de características

In [ ]:
# Seleccionar características importantes
selector = SelectFromModel(best_rf, threshold='median')
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Obtener las características seleccionadas
selected_features = X.columns[selector.get_support()]
print(f'Número de características seleccionadas: {len(selected_features)}')
print('Características seleccionadas:')
print(selected_features.tolist())

## Guardar el modelo

In [ ]:
# Guardar el modelo entrenado y el scaler
with open('../models/modelo_breast_cancer.pkl', 'wb') as archivo:
    pickle.dump(best_rf, archivo)

with open('../models/scaler_breast_cancer.pkl', 'wb') as archivo:
    pickle.dump(scaler, archivo)

print('Modelo y scaler guardados correctamente.')